In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('goibibo_flights_data.csv')

In [3]:
df.head()

,flight date,airline,flight_num,class,from,dep_time,to,arr_time,duration,price,stops,Unnamed: 11,Unnamed: 12
0,26-06-2023,SpiceJet,SG-8709,economy,Delhi,18:55,Mumbai,21:05,02h 10m,"6,013",non-stop,NaN,NaN
1,26-06-2023,SpiceJet,SG-8157,economy,Delhi,06:20,Mumbai,08:40,02h 20m,"6,013",non-stop,NaN,NaN
2,26-06-2023,AirAsia,I5-764,economy,Delhi,04:25,Mumbai,06:35,02h 10m,"6,016",non-stop,NaN,NaN
3,26-06-2023,Vistara,UK-995,economy,Delhi,10:20,Mumbai,12:35,02h 15m,"6,015",non-stop,NaN,NaN
4,26-06-2023,Vistara,UK-963,economy,Delhi,08:50,Mumbai,11:10,02h 20m,"6,015",non-stop,NaN,NaN


df['price'] = pd.to_numeric(df['price'].str.replace(',', ''), errors='coerce')
df['duration'] = pd.to_numeric(df['duration'].str.replace(',', ''), errors='coerce')
df21_hist= df.groupby('duration')['price'].value_counts().reset_index(name='count')
df21_hist_pivot= df21_hist.pivot_table(index='duration',columns='price',values='price',aggfunc='count')
df21_hist_pivot

In [4]:
df['price']=pd.to_numeric(df['price'].astype(str).str.replace(',',''),errors='coerce')

In [5]:
df['price'].unique()

array([ 6013,  6016,  6015, ..., 75852, 15225, 78262])

In [6]:
import re

def convert_duration_to_minutes(duration_str):
    """Converts a duration string in the format "Xh Ym" to total minutes.

    Args:
        duration_str (str): The duration string, e.g., "2h 30m".

    Returns:
        int: The total duration in minutes.
    """
    hours, minutes = map(int, duration_str[:-1].split('h '))
    return hours * 60 + minutes

# Filter out any duration values that do not follow the "Xh Ym" format
# A regular expression to match standard duration formats like "Xh Ym"
duration_pattern = re.compile(r"^\d+h \d+m$")

# Replace non-standard duration values with NaN, then drop these rows
# Convert 'duration' column to string type before applying the pattern matching
df['duration'] = df['duration'].astype(str).apply(
    lambda x: x if duration_pattern.match(x) else None
)
df.dropna(subset=['duration'], inplace=True)

# Apply the conversion function again on the cleaned data
df['duration_minutes'] = df['duration'].apply(convert_duration_to_minutes)

# Display the first few rows to verify the new column with total duration in minutes
df[['duration_minutes']].head()


,duration_minutes
0,130
1,140
2,130
3,135
4,140


In [7]:
df.head()

,flight date,airline,flight_num,class,from,dep_time,to,arr_time,duration,price,stops,Unnamed: 11,Unnamed: 12,duration_minutes
0,26-06-2023,SpiceJet,SG-8709,economy,Delhi,18:55,Mumbai,21:05,02h 10m,6013,non-stop,NaN,NaN,130
1,26-06-2023,SpiceJet,SG-8157,economy,Delhi,06:20,Mumbai,08:40,02h 20m,6013,non-stop,NaN,NaN,140
2,26-06-2023,AirAsia,I5-764,economy,Delhi,04:25,Mumbai,06:35,02h 10m,6016,non-stop,NaN,NaN,130
3,26-06-2023,Vistara,UK-995,economy,Delhi,10:20,Mumbai,12:35,02h 15m,6015,non-stop,NaN,NaN,135
4,26-06-2023,Vistara,UK-963,economy,Delhi,08:50,Mumbai,11:10,02h 20m,6015,non-stop,NaN,NaN,140


In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
ohe= OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [10]:
df['stops']=ohe.fit_transform(df[['stops']])

In [11]:
X= df.drop(columns=['price','from','flight date','airline','flight_num','class','dep_time','to','arr_time','duration','Unnamed: 11','Unnamed: 12'])
Y=df['price']

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [13]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.2,random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler= StandardScaler()

In [16]:
X_train_scaled =scaler.fit_transform(X_train)
X_test_scaled= scaler.transform(X_test)

In [17]:
model=LogisticRegression()

In [ ]:
model.fit(X_train_scaled,Y_train)